In [12]:
!pip install openpyxl
!pip install xlrd

Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 332.5 kB/s eta 0:00:00a 0:00:01


In [25]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import requests
import io

In [44]:
def get_lotto_history(round, file_path):
    try:
        # 인덱스를 지정해 시트 설정
        df = pd.read_excel(file_path, engine='openpyxl')
        # 필요한 컬럼 선택 및 이름 변경
        lotto_df = df.iloc[:, [0] + list(range(0, 7))]
        lotto_df.columns = ['round', '1', '2', '3', '4', '5', '6', 'bonus']
        print(f"최신 데이터 로드 완료. [{round}]")
        return lotto_df
    except requests.exceptions.RequestException as e:
        print(f"데이터를 가져오는 데 실패했습니다: {e}")
        return None


def prepare_data(df, verbose=0):
    """
    로또 데이터를 머신러닝 모델이 학습할 수 있는 형태로 변환합니다.
    (각 번호가 당첨되었는지 여부를 나타내는 45개의 컬럼 생성)
    """
    # 당첨번호 1~6만 사용
    win_numbers = df[['1', '2', '3', '4', '5', '6']]
    
    # 1부터 45까지 각 번호가 당첨되었는지를 0과 1로 표현하는 데이터프레임 생성
    y = pd.DataFrame(0, index=df.index, columns=range(1, 46))
    if verbose > 0:
        print(f'prepare_data.0 \n\t{win_numbers}')

    for index, row in win_numbers.iterrows():
        for num in row:
            if 1 <= num <= 45:
                y.loc[index, num] = 1
                
    # X는 이전 회차들의 당첨 패턴, y는 다음 회차의 결과
    # 예를 들어, 10회차를 예측하기 위해 9회차까지의 패턴을 사용
    X = y.shift(1).fillna(0) # 이전 회차 데이터로 현재 회차를 예측
    
    return X, y


def predict_numbers(X, y, verbose=0):
    """
    RandomForest 모델을 학습하고 1176회차 번호를 예측합니다.
    """
    models = {}
    
    # 1부터 45까지 각 번호에 대한 모델을 개별적으로 학습
    for num in range(1, 46):
        if verbose > 0:
            print(f"{num}번 공에 대한 모델 학습 중...")
        model = RandomForestClassifier(n_estimators=100, random_state=42, verbose=verbose)
        model.fit(X, y[num])
        models[num] = model

    # 1176회를 예측하기 위해 마지막으로 관측된 1175회차 데이터를 입력으로 사용
    last_observation = pd.DataFrame(y.iloc[-1, :]).T
    
    predictions = {}
    for num, model in models.items():
        if verbose > 0:
            print(f'{num}, {model}')
        # predict_proba는 [나오지 않을 확률, 나올 확률]을 반환
        prob = model.predict_proba(last_observation)
        if verbose > 0:
            print(f'{type(prob)}, {prob}')
        if len(prob[0]) > 1:
            predictions[num] = prob[0][1]
        else:
            predictions[num] = 0.0
        
    # '나올 확률'이 가장 높은 순서대로 6개 번호 선택
    predicted_numbers = sorted(predictions, key=predictions.get, reverse=True)[:6]
    
    return sorted(predicted_numbers)


file_names = [
    (1173, 'xlsx/1173.xlsx'),
    (1174, 'xlsx/1174.xlsx'),
    (1175, 'xlsx/1175.xlsx')
    ]
results = []
for file_name in file_names:
    # --- 코드 실행 ---
    # 1. 역대 당첨 번호 데이터 가져오기
    lotto_history_df = get_lotto_history(round=file_name[0], file_path=file_name[1])
    # print(lotto_history_df)
    if lotto_history_df is not None:
        # 2. 데이터 전처리
        X_train, y_train = prepare_data(lotto_history_df)

        # 3. 모델 학습 및 예측
        print("\nstart to predict...")
        predicted_lotto_numbers = predict_numbers(X_train, y_train, verbose=0)
        results.append((file_name[0], predicted_lotto_numbers))

        print("\n--- last result ---")
        print(f"Predict based on RandomForest: {predicted_lotto_numbers}")

for result in results:
        print(result)

최신 데이터 로드 완료. [1173]

start to predict...

--- last result ---
Predict based on RandomForest: [2, 12, 16, 24, 37, 39]
최신 데이터 로드 완료. [1174]

start to predict...

--- last result ---
Predict based on RandomForest: [2, 12, 16, 19, 24, 39]
최신 데이터 로드 완료. [1175]

start to predict...

--- last result ---
Predict based on RandomForest: [2, 12, 16, 19, 24, 39]
(1173, [2, 12, 16, 24, 37, 39])
(1174, [2, 12, 16, 19, 24, 39])
(1175, [2, 12, 16, 19, 24, 39])
